In [1]:
!pip install gymnasium==0.29.1 pygame==2.3.0 pettingzoo==1.24.3 tianshou==0.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 847.8/847.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.1/163.1 kB 12.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1

In [2]:
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U accelerate jupyter ipywidgets
!pip install gymnasium
import gymnasium as gym
import re
import math
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    GenerationConfig,
    pipeline,
)
import torch
from transformers import pipeline
import csv
import datetime

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 13.0 MB/s eta 0:00:00


In [3]:
## Setup LLM Agent

In [46]:
from typing import Any, Dict

import numpy as np
from tianshou.data import Batch
from tianshou.policy import BasePolicy
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from gymnasium.spaces import Discrete, Space
from tianshou.env import PettingZooEnv
from tianshou.policy import BasePolicy, DQNPolicy
from tianshou.utils.net.common import Net
from torch import nn

class LLMAgent(BasePolicy):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        models_dict = {"StableLM Zephyr 3B": "stabilityai/stablelm-zephyr-3b"}
        model_id = models_dict["StableLM Zephyr 3B"]
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)#, device=0)
        self.pipe = pipeline("text-generation", model=model_id, device_map="auto", tokenizer=self.tokenizer, torch_dtype=torch.bfloat16)


    def field_to_string(self, field, c):
      return_value = "";
      if field[0] == 1:
        return_value = "X "
      elif field[1] == 1:
          return_value = "O "
      else:
          return_value = str(c) + " "
      if c < 6:
          return_value += "| "
      return return_value

    def board_to_string(self, batch: Batch) -> str:
        rows = batch.obs['obs']
        row1 = self.field_to_string(rows[0][0][0], 0) + self.field_to_string(rows[0][1][0], 3) + self.field_to_string(rows[0][2][0], 6)
        row2 = self.field_to_string(rows[0][0][1], 1) + self.field_to_string(rows[0][1][1], 4) + self.field_to_string(rows[0][2][1], 7)
        row3 = self.field_to_string(rows[0][0][2], 2) + self.field_to_string(rows[0][1][2], 5) + self.field_to_string(rows[0][2][2], 8)
        return row1 + '\n' + row2 + '\n' + row3

    def ask_llm_for_choice(self, board: str, possible_choices: str) -> int:
        print(board)
        self.job_description = "You will be provided with a tic tac toe board. There are two players, X and O. An empty board looks likes this:\n0 3 6\n1 4 7\n2 5 8\nWhen a player made a move a X or O is placed on the board.\nYou are player X and should choose the best possible option. The possible numbers are " + possible_choices + ". Only answer best number to choose, no comments or explanation, just a number."
        output = self.generate(board,0.7,15)
        print("Output: " + output)

        extracted_choice = [int(i) for i in re.sub(r'[^0-9\s]', '', output).split() if i.isdigit()]
        for n in extracted_choice:
          print("extracted_choice: " + str(n))
        if extracted_choice:
          print("Choice: " + str(extracted_choice[0]))
          return extracted_choice[0]
        else:
          print("No choice, try "+str(possible_choices[0]))
          return int(possible_choices[0])

    def generate(self, question, temperature=0.7, max_new_tokens=512):
        #return "left"
        messages = [
            {
                "role": "system",
                "content": self.job_description,
            },
        ]
        messages.append({"role": "user", "content": question})
        prompt = self.pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        outputs = self.pipe(prompt, max_new_tokens=max_new_tokens, do_sample=True, temperature=temperature, top_k=50, top_p=0.95)
        output = outputs[0]["generated_text"]
        messages.append({"role": "assistant", "content": output})
        #print(output)
        response_start = output.rfind('<|assistant|>')
        text_output = output[response_start + len('<|assistant|>'):]
        return text_output

    def forward(
        self, batch: Batch, state: dict | Batch | np.ndarray | None = None
    ) -> Batch:
        all_choices = np.array(list(range(0,9)))

        board = self.board_to_string(batch)

        mask = batch.obs.mask.flatten()
        masked_choices = all_choices[mask.astype(bool)].astype(str)
        masked_choices_text = ", ".join(masked_choices)

        if len(masked_choices) == 1:
          print("Only one option: " + masked_choices[0])
          return Batch(act=[int(masked_choices[0])])

        choice = -1
        tries = 3
        while str(choice) not in masked_choices and tries > 0:
          print("try: " + str(tries))
          choice = self.ask_llm_for_choice(board, masked_choices_text)
          tries -= 1

        return Batch(act=[choice])

    def learn(self, batch: Batch) -> Dict[str, Any]:
        return {}


## Create Environment

In [47]:
from tianshou.policy import MultiAgentPolicyManager, RandomPolicy
from pettingzoo.classic import tictactoe_v3

env = PettingZooEnv(tictactoe_v3.env())

## Create Agents

In [48]:
agent = LLMAgent() # or create_dqn_agent(env)

policies = [agent, RandomPolicy()]
policy = MultiAgentPolicyManager(policies=policies, env=env)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Play


In [49]:
from tianshou.env import DummyVectorEnv, PettingZooEnv
from pettingzoo.classic import tictactoe_v3
from tianshou.data import Collector

env = DummyVectorEnv([lambda: PettingZooEnv(tictactoe_v3.env())])
collector = Collector(policy=policy, env=env)

In [53]:
result = collector.collect(n_episode=100)

try: 3
0 | 3 | 6 
1 | 4 | 7 
2 | 5 | 8 
Output: 
Player X should choose the number 3.
extracted_choice: 3
Choice: 3
try: 3
0 | X | 6 
1 | 4 | O 
2 | 5 | 8 
Output: 
I will analyze the board and determine the best move for player X.

No choice, try 0
try: 3
X | X | 6 
1 | 4 | O 
2 | O | 8 
Output: 
4
extracted_choice: 4
Choice: 4
try: 3
X | X | 6 
1 | X | O 
2 | O | O 
Output: 
The best possible option for player X is to place their next move on number
No choice, try 1
try: 3
0 | 3 | 6 
1 | 4 | 7 
2 | 5 | 8 
Output: 
Player X should choose number 3.
extracted_choice: 3
Choice: 3
try: 3
0 | X | 6 
O | 4 | 7 
2 | 5 | 8 
Output: 
X should choose the number 6.
extracted_choice: 6
Choice: 6
try: 3
0 | X | X 
O | 4 | 7 
O | 5 | 8 
Output: 
4
extracted_choice: 4
Choice: 4
try: 3
0 | X | X 
O | X | O 
O | 5 | 8 
Output: 
5
extracted_choice: 5
Choice: 5
try: 3
0 | 3 | 6 
1 | 4 | 7 
2 | 5 | 8 
Output: 
Player X should choose the number 3.
extracted_choice: 3
Choice: 3
try: 3
0 | X | 6 
1 | O | 7 

In [54]:
won = 0
draw = 0
lost = 0
for res in result['rews']:
  if res[0] == 1:
    won += 1
  elif res[0] == -1:
    lost +=1
  else:
    draw += 1

print("Win: " + str(won) + " lost: " + str(lost) + " draw: " + str(draw))

Win: 62 lost: 34 draw: 4
